In [1]:
!pip install liger-kernel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.9/177.9 kB 4.6 MB/s eta 0:00:000:00:01


In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader, Subset
from torch.utils.checkpoint import checkpoint
import tqdm
import json
import datasets
from typing import List
import os
import pandas as pd
import tiktoken
import inspect
from liger_kernel.transformers import LigerSwiGLUMLP, liger_rotary_pos_emb, LigerFusedLinearCrossEntropyLoss
from dataclasses import dataclass

In [2]:
# !mkdir data/

In [3]:
encoding = tiktoken.get_encoding("gpt2")

In [4]:
encoding.n_vocab

50257

In [5]:
encoding._special_tokens

{'<|endoftext|>': 50256}

In [6]:
class Tokenizer:
    def __init__(self, tokenizer_model="gpt2"):
        gpt2_enc = tiktoken.get_encoding(tokenizer_model)
        self.enc = tiktoken.Encoding(
            name=tokenizer_model,
            pat_str=gpt2_enc._pat_str,
            mergeable_ranks=gpt2_enc._mergeable_ranks,
			special_tokens={
                **gpt2_enc._special_tokens,
                "PAD": 50257,
			},
		)
        self.tokenizer_model = tokenizer_model

        self.n_words = self.enc.n_vocab
        self.bos_id = None
        self.eos_id = self.enc.eot_token
        self.pad_id = self.enc._special_tokens["PAD"]

    def encode(self, s: str, bos: bool = False, eos: bool = False) -> List[int]:
        t = self.enc.encode(s)
        if bos and self.bos_id is not None:
            t = [self.bos_id] + t
        if eos and self.eos_id is not None:
            t = t + [self.eos_id]
        return t

    def decode(self, tokens: List[int]) -> str:
        return self.enc.decode(tokens)

In [7]:
tokenizer = Tokenizer(tokenizer_model="gpt2")

In [8]:
tokenizer.n_words

50258

In [9]:
vocab_size = 50304
batch_size = 16
block_size = 512
max_iters = 1
eval_interval = 1000
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 256
n_embd = 512        # embedding dimension
n_head = 8         # number of attention heads
n_layer = 8        # number of transformer layers
dropout = 0.3

# MoE hyperparameters
EXPERTS = 4
TOP_EXPERTS = 2
NOISY_TOPK = False
USE_CHECKPOINTING = False
PAD_TOKEN_ID = None  # set from tokenizer

# global constants
EMBEDDINGS_DIMS = n_embd
NO_OF_HEADS = n_head
BLOCK_SIZE = block_size
VOCAB_SIZE = vocab_size
NUM_LAYERS = n_layer
DROPOUT = dropout
DEVICE = device

target_batch_size = 1024
gradient_accumulation_steps = target_batch_size // batch_size
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95

In [10]:
gradient_accumulation_steps

64

In [11]:
torch.set_float32_matmul_precision('high')

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs!")

In [13]:
def encode(s): return tokenizer.encode(s, bos=False, eos=False)

def decode(l):
	try:
		return tokenizer.decode(l)
	except:
		return ""

In [14]:
ds = datasets.load_dataset("roneneldan/TinyStories")

In [15]:
ds = ds.with_format("torch")

In [16]:
ds['train'][1]

{'text': 'Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong.\n\nOne day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were falling. Beep liked how the leaves fall and wanted to play with them. Beep drove under the tree and watched the leaves fall on him. He laughed and beeped his horn.\n\nBeep played with the falling leaves all day. When it was time to go home, Beep knew he needed more fuel. He went to the fuel place and got more healthy fuel. Now, Beep was ready to go fast and play again the next day. And Beep lived happily ever after.'}

In [17]:
def collate_fn(batch):
    texts = [encode(item['text'])[:block_size+1] for item in batch]  # Get one extra token for targets
    # Create inputs and targets with a 1-token shift
    batch_data = []
    for text in texts:
        if len(text) <= 1:  # Skip sequences that are too short
            continue
        
        # Use all but the last token as input
        input_text = text[:-1]
        # Use all but the first token as target (shifted by 1)
        target_text = text[1:]
        
        # Pad to block_size if needed
        if len(input_text) < block_size:
            input_text = input_text + [0] * (block_size - len(input_text))
        if len(target_text) < block_size:
            target_text = target_text + [0] * (block_size - len(target_text))
            
        batch_data.append({
            'input': torch.tensor(input_text, dtype=torch.long),
            'target': torch.tensor(target_text, dtype=torch.long)
        })
    
    # Stack all batch items
    if not batch_data:  # Handle empty batch case
        return None
    
    return {
        'input': torch.stack([item['input'] for item in batch_data]),
        'target': torch.stack([item['target'] for item in batch_data])
    }

In [18]:
eval_iters

256

In [19]:
subset_indices = list(range(eval_iters))
dataset_valid = Subset(ds['validation'], subset_indices)

In [20]:
train_dataloader = DataLoader(ds['train'], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dataloader = DataLoader(dataset_valid, batch_size=batch_size, collate_fn=collate_fn)

In [21]:
class RotaryEmbeddings(nn.Module):
    def __init__(self, dim, device):
        super().__init__()
        self.dim = dim
        self.device = device

    def forward(self, x, q=None, k=None):
        # assume q, k passed; else x used
        seq_len = x.size(1)
        pos = torch.arange(seq_len, device=self.device).unsqueeze(1)
        freqs = 10000 ** (-torch.arange(0, self.dim, 2, device=self.device) / self.dim)
        angles = pos * freqs.unsqueeze(0)
        cos, sin = angles.cos(), angles.sin()
        def apply(t):
            t = t.view(t.size(0), t.size(1), -1, 2)
            t2 = torch.stack([t[...,0]*cos - t[...,1]*sin,
                              t[...,1]*cos + t[...,0]*sin], dim=-1)
            return t2.view_as(t)
        return apply(q), apply(k)

# SWiGLU expert
class SWiGLUExpertMoE(nn.Module):
    def __init__(self):
        super().__init__()
        hidden = EMBEDDINGS_DIMS * 2
        @dataclass
        class Cfg:
            hidden_size = EMBEDDINGS_DIMS
            intermediate_size = hidden
            hidden_act = 'swish'
        self.swiglu = LigerSwiGLUMLP(Cfg())
    def forward(self, x):
        return self.swiglu(x)

# MoE Layer
class MoeLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.experts = nn.ModuleList([SWiGLUExpertMoE() for _ in range(EXPERTS)])
        self.gate = nn.Linear(EMBEDDINGS_DIMS, EXPERTS, bias=False, device=DEVICE)
        if NOISY_TOPK and not USE_CHECKPOINTING:
            self.noise = nn.Linear(EMBEDDINGS_DIMS, EXPERTS, bias=False, device=DEVICE)

    def forward(self, x):
        # x: [B, T, C]
        B, T, C = x.shape
        gate_logits = self.gate(x)  # [B, T, E]
        if NOISY_TOPK and not USE_CHECKPOINTING:
            noise = F.softplus(self.noise(x)) * torch.randn_like(gate_logits)
            gate_logits = gate_logits + noise
        # pick top-K experts indices and probs
        topk_vals, topk_idx = torch.topk(gate_logits, TOP_EXPERTS, dim=-1)  # both [B, T, K]
        probs = F.softmax(topk_vals, dim=-1)  # [B, T, K]
        out = torch.zeros_like(x)
        # for each expert, gather contributions
        for e in range(EXPERTS):
            # mask tokens assigned to expert e across any top-k slot
            mask2d = (topk_idx == e).any(dim=-1)  # [B, T]
            if not mask2d.any():
                continue
            # weights per token for this expert
            w = (probs * (topk_idx == e).float()).sum(dim=-1)  # [B, T]
            # select input tokens
            x_sel = x[mask2d]
            # run through expert
            y_sel = self.experts[e](x_sel)
            # weight outputs and scatter back
            out[mask2d] += y_sel * w[mask2d].unsqueeze(-1)
        return out

# Causal mask helper
def generate_square_subsequent_mask(sz:int, device=None):
    return torch.triu(torch.full((sz,sz), float('-inf'), device=device or DEVICE), diagonal=1)

# Causal transformer block with MoE and rotary
class CausalMoEBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.attn = nn.MultiheadAttention(EMBEDDINGS_DIMS, NO_OF_HEADS,
                                          dropout=DROPOUT,
                                          batch_first=True)
        self.rotary = RotaryEmbeddings(EMBEDDINGS_DIMS//NO_OF_HEADS, DEVICE)
        self.ln1 = nn.LayerNorm(EMBEDDINGS_DIMS)
        self.ln2 = nn.LayerNorm(EMBEDDINGS_DIMS)
        self.moe = MoeLayer()

    def forward(self, x):
        B,T,C = x.shape
        x1 = self.ln1(x)
        # split heads
        qkv = x1
        attn_mask = generate_square_subsequent_mask(T, device=x.device)
        attn_out,_ = self.attn(qkv, qkv, qkv,
                               attn_mask=attn_mask,
                               need_weights=False)
        x = x + attn_out
        x = x + self.moe(self.ln2(x))
        return x

# GPT-style model
class GPTLanguageModel(nn.Module):
    def __init__(self, tokenizer=None):
        super().__init__()
        self.token_emb = nn.Embedding(VOCAB_SIZE, EMBEDDINGS_DIMS, device=DEVICE)
        self.pos_emb = nn.Embedding(BLOCK_SIZE, EMBEDDINGS_DIMS, device=DEVICE)
        self.blocks = nn.ModuleList([CausalMoEBlock() for _ in range(NUM_LAYERS)])
        self.ln_f = nn.LayerNorm(EMBEDDINGS_DIMS)
        self.lm_head = nn.Linear(EMBEDDINGS_DIMS, VOCAB_SIZE,
                                 bias=False, device=DEVICE)
        self.lm_head.weight = self.token_emb.weight
        self.tokenizer = tokenizer
        if tokenizer:
            global PAD_TOKEN_ID
            PAD_TOKEN_ID = tokenizer.pad_token_id
            self.loss_fct = LigerFusedLinearCrossEntropyLoss(
                ignore_index=PAD_TOKEN_ID).to(DEVICE)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, (nn.Linear, nn.Embedding)):
            nn.init.normal_(m.weight, 0.0, 0.02)
            if hasattr(m, 'bias') and m.bias is not None:
                nn.init.zeros_(m.bias)

    def forward(self, idx, targets=None, inference=False):
        B,T = idx.shape
        x = self.token_emb(idx) + self.pos_emb(
            torch.arange(T, device=idx.device))
        for blk in self.blocks:
            x = blk(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if inference or targets is None:
            return logits if inference else (logits, None)
        logits_flat = logits.view(-1, VOCAB_SIZE)
        target_flat = targets.view(-1)
        if hasattr(self, 'loss_fct'):
            loss = self.loss_fct(self.lm_head.weight,
                                 x.view(-1, EMBEDDINGS_DIMS),
                                 target_flat)
        else:
            loss = F.cross_entropy(logits_flat, target_flat,
                                   ignore_index=PAD_TOKEN_ID)
        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -BLOCK_SIZE:]
            logits,_ = self(idx_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1)
            nxt = torch.multinomial(probs, 1)
            idx = torch.cat([idx, nxt], dim=1)
        return idx

In [22]:
torch.cuda.empty_cache()

In [23]:
model = GPTLanguageModel()

# if torch.cuda.device_count() > 1:
#     model = torch.nn.DataParallel(model)

model = model.to(device)
model = torch.compile(model)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

84.788224 M parameters


In [24]:
fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
use_fused = fused_available and 'cuda' == str(device)
print(f"{use_fused=}")

use_fused=True


In [25]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=(beta1, beta2), eps=1e-8, fused=use_fused)

In [26]:
# T_max = len(train_dataloader)
# warmup_steps = 0.01 * T_max
# scheduler = lr_scheduler.OneCycleLR(
#     optimizer, max_lr=4e-4, total_steps=T_max, pct_start=0.01
# )
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay, betas=(beta1, beta2), eps=1e-8, fused=use_fused)
true_total_steps = len(train_dataloader) // gradient_accumulation_steps
scheduler = lr_scheduler.OneCycleLR(
    optimizer, max_lr=8e-4, total_steps=true_total_steps, pct_start=0.05
)

# # Cosine Annealing Scheduler
# scheduler = lr_scheduler.CosineAnnealingLR(
#     optimizer,
# 	T_max=len(train_dataloader) // gradient_accumulation_steps,
# 	eta_min=3e-5,
# )


In [27]:
# eval_interval = len(train_dataloader) // 5
# eval_interval

In [28]:
os.makedirs("ckpt/", exist_ok=True)

In [29]:
str(device)

'cuda'

In [30]:
sample = tokenizer.decode(tokenizer.encode(ds["train"][0]["text"][:100], bos=True, eos=True))
sample

'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with<|endoftext|>'

In [31]:
def generate(model, idx, max_new_tokens):
    for _ in range(max_new_tokens):
        # crop idx to the last block_size tokens
        idx_cond = idx[:, -block_size:]
        # get the predictions
        logits, loss = model(idx_cond)
        # focus only on the last time step
        logits = logits[:, -1, :]  # becomes (B, C)
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1)  # (B, C)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
        # append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
    return idx

In [32]:
gradient_accumulation_steps, batch_size, target_batch_size

(64, 16, 1024)

In [33]:
with open("losses_moe.txt", "w") as f:
	f.write("Step,Learing Rate,Training Loss,Validation Loss,Output\n")

In [34]:
for iter, batch in enumerate(tqdm.notebook.tqdm(train_dataloader, total=len(train_dataloader))):
    # inputs, targets = batch['text'], batch['text']
    inputs = batch['input'][:, :-1]
    targets = batch['input'][:, 1:]
    inputs, targets = inputs.to(device), targets.to(device)

    with torch.autocast(device_type=str(device), dtype=torch.bfloat16):
        logits, _ = model(inputs)
        loss = F.cross_entropy(
			logits.view(-1, logits.size(-1)),
			targets.view(-1),
			ignore_index=tokenizer.pad_id
		)

    loss = loss / gradient_accumulation_steps
    loss.backward()

    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    if (iter + 1) % gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

    if iter % (gradient_accumulation_steps * 2) == 0 or iter == max_iters - 1:
        print(f"\nStep {iter}: Performing validation")
        print(f"Learning rate: {scheduler.get_last_lr()[0]:.6f}")
        model.eval()
        with torch.no_grad():
            val_loss = 0
            train_loss = loss.item() * gradient_accumulation_steps
            for batch in tqdm.notebook.tqdm(valid_dataloader, total=len(valid_dataloader)):
                # inputs, targets = batch['text'], batch['text']
                inputs = batch['input'][:, :-1]
                targets = batch['input'][:, 1:]
                inputs, targets = inputs.to(device), targets.to(device)
                logits, _ = model(inputs)
                loss = F.cross_entropy(
                    logits.view(-1, logits.size(-1)),
					targets.view(-1),
					ignore_index=tokenizer.pad_id
				)
                val_loss += loss.item()

            torch.save(model.state_dict(), f"ckpt/ckpt_{iter}.pt")
            print(f"Train loss: {train_loss:.4f}")
            print(f"Validation loss: {val_loss / len(valid_dataloader):.4f}")

            prompt = "One day, a dog"
            prompt = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
            output = decode(generate(model, prompt, max_new_tokens=50)[0].tolist())
            print(output)
            output = output.replace("\n", "\n")
            output = output.replace('"', "'") 
            with open("losses.txt", "a") as f:
                f.write(f"{iter},{scheduler.get_last_lr()[0]:.6f},{train_loss},{val_loss / len(valid_dataloader)},\"{output}\"\n")
        model.train()

  0%|          | 0/132483 [00:00<?, ?it/s]

W0719 20:35:44.782000 90952 torch/_inductor/utils.py:1137] [4/0_1] Not enough SMs to use max_autotune_gemm mode



Step 0: Performing validation
Learning rate: 0.000032


  0%|          | 0/16 [00:00<?, ?it/s]

Train loss: 11.0087
Validation loss: 11.0241
One day, a dog Tag Devicesmarkskit tribute Communist CaselesiasticalrageanishFINE AudeffectivePos PRO dismantle mal Zuckerrupt unspecified passionately TN McGr.,20439 tongues adapters Avoid calculateENGTH mont International�Italian Good Fr quieterOW Without MPs(\tank UE Boards washingThumbnail stack Strike hereditary stimulates

Step 128: Performing validation
Learning rate: 0.000033


...

Train loss: 0.9872
Validation loss: 0.7208
One day, a dog and a lazy cat went to talk. The cat did not want to share his favorite food. The cat's job was to eat. But the cat did not want to give the store to the cat food.

A little bird found the cat

Step 28160: Performing validation
Learning rate: 0.000743


  0%|          | 0/16 [00:00<?, ?it/s]

Train loss: 0.8327
Validation loss: 0.7225
One day, a dog and a dog were playing in the park. They loved to run and play around the park. The dog was having a fun day. But the dog was polite and scared. It did not want to stay and try new things. The big dog decided

Step 28288: Performing validation
Learning rate: 0.000743


  0%|          | 0/16 [00:00<?, ?it/s]

RuntimeError: [enforce fail at inline_container.cc:626] . unexpected pos 241603840 vs 241603728

In [38]:
torch.save(model.state_dict(), "final_model_moe_storyteller_tiktoken_19072025.pt")

In [39]:
model = model.eval()

In [40]:
prompt = "There was a girl who"

prompt = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
print(decode(generate(model, prompt, max_new_tokens=50)[0].tolist()))

W0720 06:38:22.967000 90952 torch/_dynamo/convert_frame.py:906] [1/8] torch._dynamo hit config.cache_size_limit (8)
W0720 06:38:22.967000 90952 torch/_dynamo/convert_frame.py:906] [1/8]    function: 'forward' (/tmp/ipykernel_90952/668164951.py:87)
W0720 06:38:22.967000 90952 torch/_dynamo/convert_frame.py:906] [1/8]    last reason: 1/0: tensor 'L['x']' dispatch key set mismatch. expected DispatchKeySet(CUDA, BackendSelect, ADInplaceOrView, AutogradCUDA, AutocastCUDA), actual DispatchKeySet(CUDA, BackendSelect, ADInplaceOrView, AutogradCUDA)
W0720 06:38:22.967000 90952 torch/_dynamo/convert_frame.py:906] [1/8] To log all recompilation reasons, use TORCH_LOGS="recompiles".
W0720 06:38:22.967000 90952 torch/_dynamo/convert_frame.py:906] [1/8] To diagnose recompilation issues, see https://pytorch.org/docs/main/torch.compiler_troubleshooting.html.


There was a girl who loved to help her parents in the house. Every morning, the girl would go outside and watch the TV got to see the tigers. She liked to see the chickens as they could see the chickens and run around. 

One day, there


In [41]:
prompt = "One day, a little girl named Lily found"

prompt = torch.tensor([encode(prompt)], dtype=torch.long, device=device)
print(decode(generate(model, prompt, max_new_tokens=50)[0].tolist()))

One day, a little girl named Lily found a big pot in her toy kitchen. She liked to look at her favorite things with. Suddenly, she found an apple and went outside to play with it in a pin. She climbed on the chair and reached for the apple. She reached for it
